In [ ]:
#https://www.youtube.com/watch?v=ntGrOburPdQ

import pandas as pd
from binance.client import Client

In [ ]:
client = Client()
cols = [
    "Time"
    ,"Open"
    ,"High"
    ,"Low"
    ,"Close"
    ,"Volume"
]
symbol = "BTCUSDT"
data_interval = "1h"
start_time = "2023-01-01"
return_threshold = .01
stop_loss = .98
trading_fee = .0015
budget = 500

In [ ]:
def get_coin_historical_data(symbol, start_time, data_interval, df_columns):

    frame = pd.DataFrame(client.get_historical_klines(
        symbol
        ,data_interval
        ,start_time
    ))
    frame = frame.iloc[:,:len(df_columns)]
    frame.columns = df_columns
    frame["Datetime"] = pd.to_datetime(frame[df_columns[0]], unit = "ms")
    return frame

# percentage change between one price to the next
def get_returns_and_shift(df, field):
    df["Price"] = df["Open"].shift(-1)
    df[field + "Returns"] = df[field].astype(float).pct_change()
    return df.dropna()

def get_hisotrical_profits(df, return_column, return_threshold, stop_loss, trading_fee):
    profits = []
    # whether I am in a position of buying
    in_position = False

    for indes, row in df.iterrows():

        # If I am not already in a buying position and the price has raised more then the percentage I want, then set the buy price to the next available open
        if not in_position and row[return_column] > return_threshold:
            buy_price = float(row["Price"])
            in_position = True
            # set trailing stop, i.e. threshold below which you want to sell
            trailing_stop = buy_price * stop_loss
        if in_position:
            # if the price is increasing, redefine the trailing stop
            if float(row["Close"]) * stop_loss >= trailing_stop:
                trailing_stop = float(row["Close"]) * stop_loss
            if float(row["Close"]) <= trailing_stop:
                sell_price = float(row["Close"])
                profit = (sell_price - buy_price) / buy_price - trading_fee
                profits.append(profit)
                in_position = False
    return profits

def historical_absolute_profits(profits, budget):
    history = []
    for item in range(len(profits)):
        budget = budget * (1 + profits[item])
        history.append(budget)
    return history

def get_cumulative_profits(profits):
    return (pd.Series(profits) + 1).cumprod()

In [ ]:
hisotrical_data = get_coin_historical_data(
    symbol = symbol
    ,start_time = start_time
    ,data_interval = data_interval
    ,df_columns = cols
)
hisotrical_data

In [ ]:
hisotrical_data = get_returns_and_shift(hisotrical_data, "Close")
hisotrical_data.head()

In [ ]:
profits = get_hisotrical_profits(
    df = hisotrical_data
    ,return_column = "CloseReturns"
    ,return_threshold = return_threshold
    ,stop_loss = stop_loss
    ,trading_fee = trading_fee
)
profits

In [ ]:
absolute_profits = historical_absolute_profits(
    profits = profits
    ,budget = budget
)
absolute_profits

In [ ]:
cum_profits = get_cumulative_profits(profits)
cum_profits